In [ ]:
!git clone https://github.com/sosamandara/token_reduction_nlp.git
!pip install transformers datasets pandas

In [2]:
%cd /content/token_reduction_nlp/notebooks

/content/token_reduction_nlp/notebooks


In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Assuming the notebooks are in the notebooks directory and executed from there
project_root = os.path.abspath(os.path.join(current_dir, '..'))
project_root = os.path.abspath(os.path.join(project_root, '..'))

# Add the models and src directories to the Python path
models_dir = os.path.join(project_root, 'models')
src_dir = os.path.join(project_root, 'src')

sys.path.append(models_dir)
sys.path.append(src_dir)

# Verify that the paths are correctly added
print("Current directory:", current_dir)
print("Project root directory:", project_root)
print("Models directory added to sys.path:", models_dir in sys.path)
print("Src directory added to sys.path:", src_dir in sys.path)

Current directory: d:\TESI\token_reduction_nlp\src\perplexity_imdb
Project root directory: d:\TESI\token_reduction_nlp
Models directory added to sys.path: True
Src directory added to sys.path: True


In [3]:
#!pip install datasets
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from modeling_topK_gpt2 import CustomGPT2LMHeadModel
from collections import defaultdict
import matplotlib.pyplot as plt
from datasets import load_dataset
import torch
from functions import load_custom_model
import numpy as np
from tqdm import tqdm
import pandas as pd

d:\Conda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
d:\Conda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
def calculate_reduction(lenght, mask):
  actual_lenght=lenght
  for percentage in mask:
    actual_lenght = (actual_lenght - int(actual_lenght*percentage))
  return actual_lenght + 1

window_sizes = [50, 100, 200, 300, 500, 1000]
remaining_tokens = []
per_keep = []

mask_30 = [0.0, 0.0, 0.0, 0.0, 0.72, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mask_50 = [0.0, 0.0, 0.0, 0.0, 0.52, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mask_70 = [0.0, 0.0, 0.0, 0.0, 0.32, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mask_90 = [0.0, 0.0, 0.0, 0.0, 0.12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

masks = [mask_30, mask_50, mask_70, mask_90]
mask_number = ["30","50","70","90"]

for i, mask in enumerate(masks):
  #remaining_tokens = []
  per_keep = []
  for w in window_sizes:
    #remaining_tokens.append(calculate_reduction(w, mask))
    per_keep.append(round(calculate_reduction(w, mask)/w,2))
    #print(remaining_tokens)
  print("-----",mask_number[i],"-----")
  print(per_keep)

----- 30 -----
[0.3, 0.29, 0.28, 0.28, 0.28, 0.28]
----- 50 -----
[0.5, 0.49, 0.48, 0.48, 0.48, 0.48]
----- 70 -----
[0.7, 0.69, 0.69, 0.68, 0.68, 0.68]
----- 90 -----
[0.9, 0.89, 0.89, 0.88, 0.88, 0.88]


In [6]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
config = GPT2Config.from_pretrained(model_name)
model_gpt = GPT2LMHeadModel.from_pretrained(model_name, config=config).to('cuda')

# Layers to prune
layers_to_prune = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

# Load custom models with different k_percent values
custom_models = {}
for i, mask in enumerate(masks):
    model_id = f"topK_{mask_number[i]}"
    custom_models[model_id] = load_custom_model(model_name, config, mask, selection_method="top_k", layers_to_prune=layers_to_prune)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
# function to generate text
def generate_text(model, tokenizer, input_text, flag=False):
    # tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    max_length = len(tokenizer(input_text)["input_ids"])+1
    if flag:
        max_length = len(tokenizer(input_text)["input_ids"])+50
    # generate text
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)

    # decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

input_text = """This film features two of my favorite guilty pleasures. Sure, the effects are laughable, the story confused, but just watching Hasselhoff in his Knight Rider days is always fun. I especially like the old hotel they used to shoot this in, it added to what little suspense was mustered. Give it a 3."""

In [ ]:
input_text = """I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to always say "Gene Roddenberry's Earth..." otherwise people would not continue watching. Roddenberry's ashes must be turning in their orbit as this dull, cheap, poorly edited (watching it without advert breaks really brings this home) trudging Trabant of a show lumbers into space. Spoiler. So, kill off a main character. And then bring him back as another actor. Jeeez! Dallas all over again.	"""
for _ in range(50):
    input_text = generate_text(model_gpt, tokenizer, input_text)
input_text

'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a\'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they have 

In [ ]:
input_text = """I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to always say "Gene Roddenberry's Earth..." otherwise people would not continue watching. Roddenberry's ashes must be turning in their orbit as this dull, cheap, poorly edited (watching it without advert breaks really brings this home) trudging Trabant of a show lumbers into space. Spoiler. So, kill off a main character. And then bring him back as another actor. Jeeez! Dallas all over again.	"""
for _ in range(50):
    input_text = generate_text(custom_models["topK_90"], tokenizer, input_text)
input_text

'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a\'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they have 

In [ ]:
input_text = """I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to always say "Gene Roddenberry's Earth..." otherwise people would not continue watching. Roddenberry's ashes must be turning in their orbit as this dull, cheap, poorly edited (watching it without advert breaks really brings this home) trudging Trabant of a show lumbers into space. Spoiler. So, kill off a main character. And then bring him back as another actor. Jeeez! Dallas all over again.	"""
for _ in range(50):
    input_text = generate_text(custom_models["topK_30"], tokenizer, input_text)
input_text

'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a\'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they have 

In [8]:
def evaluate_perplexity_for_query(model, tokenizer, encodings, input_text, num_generations=50):

    generated_text = input_text
    i = 0
    max_length = len(tokenizer(generated_text)["input_ids"])
    while i<num_generations and max_length<1024:
      generated_text = generate_text(model, tokenizer, generated_text)
      i = i+1
      max_length = max_length+1

    input_ids = encodings.input_ids.to('cuda')
    seq_len = input_ids.size(1)
    log_probs = []
    predictions = []

    for i in range(seq_len - 1):
        with torch.no_grad():
            output = model(input_ids[:, :i + 1])
            logits = output.logits[:, -1, :]  # Get logits for the last token
            probs = torch.nn.functional.log_softmax(logits, dim=-1)
            target_token_id = input_ids[0, i + 1].item()
            log_prob = probs[0, target_token_id].item()
            log_probs.append(log_prob)

            # Get the predicted token
            predicted_token_id = torch.argmax(probs, dim=-1).item()
            predictions.append(predicted_token_id)

    avg_log_prob = np.mean(log_probs)
    perplexity = np.exp(-avg_log_prob)
    return perplexity, generated_text[len(input_text):]

def evaluate_perplexity_for_each_query(model, tokenizer, dataset):
    results = []
    for text in tqdm(dataset['text'][:100]):  # Process only the first 10 entries for now
        encodings = tokenizer(text, return_tensors='pt')
        perplexity, predicted_text = evaluate_perplexity_for_query(model, tokenizer, encodings, text)
        results.append({
            'query': text,
            'perplexity': perplexity,
            'prediction': predicted_text
        })
    return results


In [9]:
imdb_dataset = load_dataset('imdb', split='test')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
results = {}
models = [model_gpt] + list(custom_models.values())
model_names = ['gpt2'] + [f'custom_{mask_number[i]}%' for i in range(len(custom_models))]

# Evaluate and save perplexities for the first 10 queries
all_results = []
for model, model_name in zip(models, model_names):
    print(f"Evaluating perplexity for each query using {model_name}")
    model_results = evaluate_perplexity_for_each_query(model, tokenizer, imdb_dataset)
    for result in model_results:
        result['model'] = model_name
    all_results.extend(model_results)

# Convert the results to a DataFrame and display
df_results = pd.DataFrame(all_results)
print(df_results)


# Optionally save to CSV
output_path = 'perplexity_results_imdb_first_100_3.csv'
df_results.to_csv(output_path, index=False)

print(f"Saved perplexity results to {output_path}")

# Display the first few results for inspection
df_results.head()

Evaluating perplexity for each query using gpt2


100%|██████████| 100/100 [14:02<00:00,  8.42s/it]


Evaluating perplexity for each query using custom_30%


100%|██████████| 100/100 [13:24<00:00,  8.04s/it]


Evaluating perplexity for each query using custom_50%


...

100%|██████████| 100/100 [15:44<00:00,  9.45s/it]


Evaluating perplexity for each query using custom_90%


100%|██████████| 100/100 [17:27<00:00, 10.47s/it]

                                                 query  perplexity  \
0    I love sci-fi and am willing to put up with a ...   44.072236   
1    Worth the entertainment value of a rental, esp...   45.943925   
2    its a totally average film with a few semi-alr...   55.653095   
3    STAR RATING: ***** Saturday Night **** Friday ...   48.609417   
4    First off let me say, If you haven't enjoyed a...   28.907190   
..                                                 ...         ...   
495  This film seems well made, and more efforts sh...   43.413178   
496  It hurt to watch this movie, it really did... ...   56.897496   
497  Rita Hayworth is just stunning at times and, f...   38.710037   
498  Like 'Singin' in the Rain', 'Cover Girl' has a...   26.915310   
499  Rita Hayworth plays a Brooklyn nightclub dance...  103.827074   

                                            prediction       model  
0    \n\nI'm not saying that Babylon 5 is bad. I'm ...        gpt2  
1                    

,query,perplexity,prediction,model
0,I love sci-fi and am willing to put up with a ...,44.072236,\n\nI'm not saying that Babylon 5 is bad. I'm ...,gpt2
1,"Worth the entertainment value of a rental, esp...",45.943925,5 stars.,gpt2
2,its a totally average film with a few semi-alr...,55.653095,".\n\n\nOverall, I would recommend this film to...",gpt2
3,STAR RATING: ***** Saturday Night **** Friday ...,48.609417,* * * * * * * * * * * * * * * * * * * * * * *...,gpt2
4,"First off let me say, If you haven't enjoyed a...",28.907190,"\n\n\nI have to say, I am a huge fan of the mo...",gpt2


In [ ]:
df_results = pd.read_csv("perplexity_results_imdb_first_100.csv")

In [ ]:
# Ensure 'index' column is present
df_results.reset_index(drop=True, inplace=True)

# Create a new DataFrame to hold the results with perplexity ratios
df_ratios = df_results.copy()

# Calculate the perplexity ratio for custom models compared to GPT-2
gpt2_perplexities = df_results.loc[0:99, 'perplexity'].values
mask_number = ["30", "50", "70", "90"]
for i, mask in enumerate(mask_number):
    start = (i + 1) * 100
    end = start + 100
    custom_model_name = f'custom_{mask}%'
    df_ratios.loc[start:end-1, 'perplexity_ratio'] = df_results.loc[start:end-1, 'perplexity'].values / gpt2_perplexities

# Save the updated DataFrame to a new CSV file
output_path = 'perplexity_results_imdb_first_100_with_ratio_3.csv'
df_ratios.to_csv(output_path, index=False)

print(f"Saved perplexity results to {output_path}")

# Display the first few results for inspection
df_ratios.head(20)

In [13]:
df_ratios.to_csv("perplexity_results_imdb_first_100_with_ratio_3.csv", index=False)

In [35]:
############
df_ratios = pd.read_csv("perplexity_results_imdb_first_100_with_ratio.csv")

In [69]:

df_ratios.reset_index(drop=True, inplace=True)

# Function to find the top-N and bottom-N rows based on perplexity ratio for a given model
def find_extremes(df, model_name, n=2):
    df_model = df[df['model'] == model_name]
    top_n_rows = df_model.nlargest(n, 'perplexity_ratio')
    bottom_n_rows = df_model.nsmallest(n, 'perplexity_ratio')
    return top_n_rows, bottom_n_rows

# Initialize a list to store the results
extreme_rows = []

# Number of examples to retrieve for each model
n_examples = 1

# Iterate through each custom model to find the extremes
mask_number = ["30", "50", "70", "90"]
for mask in mask_number:
    custom_model_name = f'custom_{mask}%'
    top_rows, bottom_rows = find_extremes(df_ratios, custom_model_name, n=n_examples)
    for _, row in top_rows.iterrows():
        if pd.isna(row['prediction']):
            continue
        extreme_rows.append(('max', custom_model_name, row))
    for _, row in bottom_rows.iterrows():
        if pd.isna(row['prediction']):
            continue
        extreme_rows.append(('min', custom_model_name, row))

# Create a DataFrame to display the extreme rows
df_extremes = pd.DataFrame([{
    'extreme': extreme,
    'model': model,
    'query': row['query'],
    'perplexity': row['perplexity'],
    'prediction': row['prediction'].split('.')[0] + '.' if '.' in row['prediction'] else row['prediction'],
    'perplexity_ratio': row['perplexity_ratio']
} for extreme, model, row in extreme_rows])

# Check for NaN values in the prediction column and handle them
df_extremes = df_extremes.dropna(subset=['prediction'])


# Optionally save to CSV
output_extremes_path = 'perplexity_extremes_imdb.csv'
df_extremes.to_csv(output_extremes_path, index=False)

print(f"Saved extremes results to {output_extremes_path}")

Saved extremes results to perplexity_extremes_imdb.csv


In [70]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 100)  # Set display width
df_extremes

,extreme,model,query,perplexity,prediction,perplexity_ratio
0,max,custom_30%,Widow hires a psychopath as a handyman. Sloppy film noir thriller which doesn't make much of its tension promising set-up. (3/10),489.718137,\n\n\nThe Last of Us: The Fall (3.,4.655061
1,min,custom_30%,"generally speaking I don't make negative comments on here. But since this is a festival piece, I don't want you to waste your time when you could see something else that might not be playing again.<br /><br />I thought the actors were pretty bad. For instance, they totally didn't play off each other, rather, they waited to RECITE their lines which were pretty poor to begin with. The dialogue sounded really forced. Norman or whatever his name tried, or so it would appear, to be witty and biting in the lines he chose but just fell really short.<br /><br />After words he asked if anyone saw the ending coming and some people were all ""yea"", and he all but called them liars. Look there were so many clues, the biggest being a briefcase full of cash for a $500 an hour whore. I mean the john gave her at least 20g's... tell tale sign. Now no you couldn't see exactly what was going to happen but by the time the twist actually occurred, I for one, didn't even care. I was just glad to get out of there. I asked him which draft he shot and he said 8.1, maybe next time he will wait to shoot 'til 15.3 cause this needed a lot of work.<br /><br />But he seemed like a fairly nice guy, he is making his own films, he'll probably get better and I hope he does, not in a snotty way either, I mean it, I wish him luck. Just remember, this is just my opinion.",45.786088,"I am not a professional, I am just a person who has been through a lot and I am not a professional.",1.144150
2,max,custom_50%,Widow hires a psychopath as a handyman. Sloppy film noir thriller which doesn't make much of its tension promising set-up. (3/10),345.566654,"\n\n\nThe Last of Us (2012)\n\n\nThe Last of Us is a film that has been around for a while, but it's been a long time coming.",3.284816
3,min,custom_50%,"Seriously, I can't imagine how anyone could find a single flattering thing to say about this movie, much less find it in themselves to write the glowing compliments contained in this comment section. How many methamphetamines was Bogdonovitch on during the filming of this movie? Was he giving a bonus to the actor that spat his lines out with the most speed and least inflection or thought? The dialogue is bad, the plot atrocious, even for a ""screwball"" comedy, and claims that the movie is an homage to classic film comedy is about the most inane thing I've ever heard. The cinematography is below the quality and innovation of that exhibited by the worst made-for-TV movies, the acting is awful (although I get the feeling that the fault for that lies squarely in the lap of the director), and speaking of which, did I mention the direction is so haphazard and inscrutable that it defies the definition of the word? The whole thing is a terribly unfunny (even in the much-beleaguered world of so-bad-it's-funny clunkers), soul-sucking, waste of two hours of your life that you'll never get back. Be afraid, be very afraid...",38.168736,\n\n\nThe only thing that really makes this movie stand out is the fact that it's a comedy.,1.070361
4,max,custom_70%,"This only gets bashed because it stars David Hasselhoff. Well, then let me bash it to. Compared to the garbage they call horror coming out nowadays, this film isn't too bad. It has the beautiful Leslie Cumming. She is super hot, but can't talk very well. There is a great scene with her when she is supernaturally raped. She shows off her nice body. Linda Blair does nothing here as well as Hasselhoff. 3/10",101.755589,\n\n\nThe film is a bit of a disappointment.,1.981920
5,min,custom_70%,"Seriously, I can't imagine how anyone could find a single flattering thing to say about this movie, much less find it in themselves to write the glowing compliments contained 

In [71]:
# Ensure the DataFrame uses its default integer index
df_ratios.reset_index(drop=True, inplace=True)

# Function to find the top-N and bottom-N rows based on perplexity ratio for a given model
def find_extremes(df, model_name, n=2):
    df_model = df[df['model'] == model_name]
    top_n_rows = df_model.nlargest(n, 'perplexity_ratio')
    bottom_n_rows = df_model.nsmallest(n, 'perplexity_ratio')
    return top_n_rows, bottom_n_rows

# Initialize a list to store the results
extreme_rows = []

# Number of examples to retrieve for each model
n_examples = 1

# Iterate through each custom model to find the extremes
mask_number = ["30", "50", "70", "90"]
for mask in mask_number:
    custom_model_name = f'custom_{mask}%'
    top_rows, bottom_rows = find_extremes(df_ratios, custom_model_name, n=n_examples)
    for _, row in top_rows.iterrows():
        if pd.isna(row['prediction']):
            continue
        extreme_rows.append(('max', custom_model_name, row))
    for _, row in bottom_rows.iterrows():
        if pd.isna(row['prediction']):
            continue
        extreme_rows.append(('min', custom_model_name, row))

# Function to cut prediction text at the second period
def cut_after_second_period(text):
    period_count = 0
    for i, char in enumerate(text):
        if char == '.':
            period_count += 1
            if period_count == 2:
                return text[:i + 1]
    return text

# Create a DataFrame to display the extreme rows
df_extremes = pd.DataFrame([{
    'extreme': extreme,
    'model': model,
    'query': row['query'],
    'perplexity': row['perplexity'],
    'prediction': cut_after_second_period(row['prediction']),
    'perplexity_ratio': row['perplexity_ratio']
} for extreme, model, row in extreme_rows])

# Check for NaN values in the prediction column and handle them
df_extremes = df_extremes.dropna(subset=['prediction'])

# Optionally save to CSV
output_extremes_path = 'perplexity_extremes_imdb.csv'
df_extremes.to_csv(output_extremes_path, index=False)

print(f"Saved extremes results to {output_extremes_path}")

Saved extremes results to perplexity_extremes_imdb.csv


In [72]:
df_extremes

,extreme,model,query,perplexity,prediction,perplexity_ratio
0,max,custom_30%,Widow hires a psychopath as a handyman. Sloppy film noir thriller which doesn't make much of its tension promising set-up. (3/10),489.718137,"\n\n\nThe Last of Us: The Fall (3.5/5)\n\n\nThe Fall is a film that's been around for a while, but it's been a long time coming.",4.655061
1,min,custom_30%,"generally speaking I don't make negative comments on here. But since this is a festival piece, I don't want you to waste your time when you could see something else that might not be playing again.<br /><br />I thought the actors were pretty bad. For instance, they totally didn't play off each other, rather, they waited to RECITE their lines which were pretty poor to begin with. The dialogue sounded really forced. Norman or whatever his name tried, or so it would appear, to be witty and biting in the lines he chose but just fell really short.<br /><br />After words he asked if anyone saw the ending coming and some people were all ""yea"", and he all but called them liars. Look there were so many clues, the biggest being a briefcase full of cash for a $500 an hour whore. I mean the john gave her at least 20g's... tell tale sign. Now no you couldn't see exactly what was going to happen but by the time the twist actually occurred, I for one, didn't even care. I was just glad to get out of there. I asked him which draft he shot and he said 8.1, maybe next time he will wait to shoot 'til 15.3 cause this needed a lot of work.<br /><br />But he seemed like a fairly nice guy, he is making his own films, he'll probably get better and I hope he does, not in a snotty way either, I mean it, I wish him luck. Just remember, this is just my opinion.",45.786088,"I am not a professional, I am just a person who has been through a lot and I am not a professional. I am just a person who has been through a lot and I am not a person who has been through a lot.",1.144150
2,max,custom_50%,Widow hires a psychopath as a handyman. Sloppy film noir thriller which doesn't make much of its tension promising set-up. (3/10),345.566654,"\n\n\nThe Last of Us (2012)\n\n\nThe Last of Us is a film that has been around for a while, but it's been a long time coming. The film is a bit of a mess, but it's still",3.284816
3,min,custom_50%,"Seriously, I can't imagine how anyone could find a single flattering thing to say about this movie, much less find it in themselves to write the glowing compliments contained in this comment section. How many methamphetamines was Bogdonovitch on during the filming of this movie? Was he giving a bonus to the actor that spat his lines out with the most speed and least inflection or thought? The dialogue is bad, the plot atrocious, even for a ""screwball"" comedy, and claims that the movie is an homage to classic film comedy is about the most inane thing I've ever heard. The cinematography is below the quality and innovation of that exhibited by the worst made-for-TV movies, the acting is awful (although I get the feeling that the fault for that lies squarely in the lap of the director), and speaking of which, did I mention the direction is so haphazard and inscrutable that it defies the definition of the word? The whole thing is a terribly unfunny (even in the much-beleaguered world of so-bad-it's-funny clunkers), soul-sucking, waste of two hours of your life that you'll never get back. Be afraid, be very afraid...",38.168736,\n\n\nThe only thing that really makes this movie stand out is the fact that it's a comedy. It's a comedy about a guy who's been in a relationship for a long time and who's been in a relationship for a long time,1.070361
4,max,custom_70%,"This only gets bashed because it stars David Hasselhoff. Well, then let me bash it to. Compared to the garbage they call horror coming out nowadays, this film isn't too bad. It has the beautiful Leslie Cumming. She is super hot, but can't talk very well. There is a great scene with her when she is supernatural

In [ ]:
selected_rows = df_ratios.iloc[[116, 216, 316, 416, 16]]

# Apply the cut_after_second_period function to the 'prediction' column
selected_rows['prediction'] = selected_rows['prediction'].apply(cut_after_second_period)

selected_rows